In [1]:
# !cp -r ../input/detectron2-newbaselineaugs ./

# !pip install detectron2-newbaselineaugs/whls/pycocotools-2.0.2/dist/pycocotools-2.0.2.tar --no-index --find-links detectron2-newbaselineaugs/whls 
# !pip install detectron2-newbaselineaugs/whls/fvcore-0.1.5.post20211019/fvcore-0.1.5.post20211019 --no-index --find-links detectron2-newbaselineaugs/whls 
# !pip install detectron2-newbaselineaugs/whls/antlr4-python3-runtime-4.8/antlr4-python3-runtime-4.8 --no-index --find-links detectron2-newbaselineaugs/whls 
# !pip install detectron2-newbaselineaugs/whls/detectron2-0.5/detectron2 --no-index --find-links detectron2-newbaselineaugs/whls

In [2]:
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-ix24ap0p
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-ix24ap0p
  Resolved https://github.com/facebookresearch/detectron2.git to commit a59f05630a8f205756064244bf5beb8661f96180
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 845.8 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.2/426.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.

In [3]:
import os
import shutil


models = os.listdir("/kaggle/input/skin-cancer-detectron-models/models")
os.makedirs("/kaggle/working/models",exist_ok = True)
for model in models:
    source = f"/kaggle/input/skin-cancer-detectron-models/models/{model}"
    destination = f"/kaggle/working/models/{model}"
    shutil.copy(source,destination)


models = os.lis

In [4]:
import torch

print(torch.__version__)


2.1.2


In [5]:
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Mon_Apr__3_17:16:06_PDT_2023
Cuda compilation tools, release 12.1, V12.1.105
Build cuda_12.1.r12.1/compiler.32688072_0
torch:  2.1 ; cuda:  2.1.2
detectron2: 0.6


In [6]:
import os, random, cv2, json, itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [7]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
# from detectron2.utils.visualizer_new import Visualizer

from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.structures import BoxMode

from sklearn.model_selection import train_test_split

In [8]:
def mask_to_polygons(mask):
    mask = np.ascontiguousarray(mask)  # some versions of cv2 does not support incontiguous arr
    res = cv2.findContours(mask.astype("uint8"), cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)
    res = res[-2]
    res = [x.flatten() for x in res]
    res = [x for x in res if len(x) >= 6]
    return res


def get_region_dicts(img_dir,mask_dir,df):  
    dataset_dicts = []
    for idx, v in tqdm(enumerate(df.name)):
        name_in_df = v
        image_name = str(v+".jpg")

        record = {}
        objs = []

        filename = os.path.join(img_dir, image_name)
        # print(filename)
        height, width = cv2.imread(filename).shape[:2]
        
        record["file_name"] = filename
        record["image_id"] = idx
        record["height"] = height
        record["width"] = width
        
        name = v
        mask_image = name+"_0_mask.jpg"
        mask = cv2.imread(mask_dir + mask_image, 0)
        ret,thresh = cv2.threshold(mask,127,255,cv2.THRESH_BINARY)

        polygons = mask_to_polygons(thresh)
        polygons = list(polygons)
        polygons.sort
        corners = polygons[-1]

        corners = [int(value) for value in corners]
        # corners = list(itertools.chain.from_iterable(corners))

        px = corners[0:][::2]
        py = corners[1:][::2]
        
        category = df.loc[df['name']==name_in_df,'target'].iloc[0]
        
        obj = {
            "bbox": [np.min(px), np.min(py), np.max(px), np.max(py)],
            "bbox_mode": BoxMode.XYXY_ABS,
            "segmentation": [corners],
            "category_id": int(category),  #0 if category=='benign' else 1,
            "iscrowd": 0
        }
        objs.append(obj)

        record["annotations"] = objs
        dataset_dicts.append(record)
            
    return dataset_dicts


In [9]:
final_train = pd.read_csv("/kaggle/input/skin-cancer-dataset/final/final_train.csv")
final_test = pd.read_csv("/kaggle/input/skin-cancer-dataset/final/final_test.csv")
final_train.shape, final_test.shape

((29675, 12), (7790, 12))

In [10]:
import glob
images = glob.glob('/kaggle/input/skin-cancer-dataset/final/images/*')
masks = glob.glob('/kaggle/input/skin-cancer-dataset/final/masks/*')
len(images), len(masks)

(37465, 37465)

In [11]:
image_dir = "/kaggle/input/skin-cancer-dataset/final/images/"
masks_dir = "/kaggle/input/skin-cancer-dataset/final/masks/"

In [12]:
for d in ["train", "test"]:
    if d == 'train':
        DatasetCatalog.register("region_final_" + d, lambda d=d: get_region_dicts(image_dir,masks_dir,final_train))
    if d == 'test':
        DatasetCatalog.register("region_final_" + d, lambda d=d: get_region_dicts(image_dir,masks_dir,final_test))
    MetadataCatalog.get("region_final_" + d).set(thing_classes=["benign","malignant"])
region_metadata = MetadataCatalog.get("region_final_train")

In [13]:
%%writefile config.yaml
SOLVER:
  OPTIMIZER: SGD
TRANSFORMERS:
 do_transform: True
 RandomBrightness:
   ENABLE: True
   MIN: 0.8
   MAX: 1.2
   PROB: 0.75
 RandomContrast:
   ENABLE: True
   MIN: 0.8
   MAX: 1.2
   PROB: 0.75
 RandomCrop:
   ENABLE: False
 RandomFlip:
   ENABLE: True
   PROB : 0.9
 RandomSaturation:
   ENABLE: True
   MIN: 0.8
   MAX: 1.2
   PROB: 0.75
 RandomLighting:
   ENABLE: True
   SCALE: 0.9
   PROB: 0.7
 RandomRotation:
   ENABLE: True
   MIN: -5
   MAX: 5
 Normalize:
   ENABLE: False
 RGBShift:
   ENABLE: False
   R_LIMIT: 20
   G_LIMIT: 20
   B_LIMIT: 20
   PROB: 0.5
 CutOutAug:
   ENABLE: False
   NUM: 5
   SIZE: 64
   PROB: 0.7
 ColorConstancyAug:
   ENABLE: True
   PROB: 0.8
 Resize:
   ENABLE: False
   HEIGHT: 768
   WIDTH: 768
 Optical:
   ENABLE: False

Writing config.yaml


In [14]:
mkdir content

In [15]:
mv config.yaml content/config.yaml

In [16]:
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
from detectron2.utils.visualizer import ColorMode
import matplotlib.pyplot as plt

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("region_final_train",)
cfg.DATASETS.TEST = ("region_final_test",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"# initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 4 #8
# cfg.SOLVER.BASE_LR = 0.003#0.00025, 0.0001, 0.001
cfg.SOLVER.MAX_ITER = 20000  # 300 iterations seems good enough, but you can certainly train longer
cfg.SOLVER.CHECKPOINT_PERIOD = 2000
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE =1024
cfg.SOLVER.IMS_PER_BATCH = 4

cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2  


# cfg.MODEL.ROI_BOX_HEAD.NUM_FC = 6
# cfg.MODEL.ROI_BOX_HEAD.FC_DIM = 512
cfg.SOLVER.BASE_LR = 0.0005   #0.00025, 0.0001

cfg.OUTPUT_DIR = "/kaggle/working/models"
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=True)
trainer.train()


[04/20 12:22:40 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

0it [00:00, ?it/s]

[04/20 12:38:19 d2.data.build]: Removed 0 images with no usable annotations. 29675 images left.
[04/20 12:38:20 d2.data.build]: Distribution of instances among all 2 categories:
|  category  | #instances   |  category  | #instances   |
|:----------:|:-------------|:----------:|:-------------|
|   benign   | 18650        | malignant  | 11025        |
|            |              |            |              |
|   total    | 29675        |            |              |
[04/20 12:38:20 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in training: [ResizeShortestEdge(short_edge_length=(640, 672, 704, 736, 768, 800), max_size=1333, sample_style='choice'), RandomFlip()]
[04/20 12:38:20 d2.data.build]: Using training sampler TrainingSampler
[04/20 12:38:20 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[04/20 12:38:20 d2.data.common]: Serializing 29675 elements to byte tensors and concatenating them all ...
[04/20 12:38:23 d2.data.

/opt/conda/lib/python3.10/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /usr/local/src/pytorch/aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


[04/20 12:39:38 d2.utils.events]:  eta: 8:13:15  iter: 12019  total_loss: 0.159  loss_cls: 0.007155  loss_box_reg: 0.008078  loss_mask: 0.1231  loss_rpn_cls: 0.00376  loss_rpn_loc: 0.003412    time: 3.5796  last_time: 2.9512  data_time: 0.0251  last_data_time: 0.0147   lr: 0.0005  max_mem: 9999M


2024-04-20 12:39:40.841508: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-20 12:39:40.841614: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-20 12:39:40.955420: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


[04/20 12:41:02 d2.utils.events]:  eta: 8:13:45  iter: 12039  total_loss: 0.1316  loss_cls: 0.007425  loss_box_reg: 0.009083  loss_mask: 0.095  loss_rpn_cls: 0.003982  loss_rpn_loc: 0.002517    time: 3.6047  last_time: 4.0590  data_time: 0.0176  last_data_time: 0.0218   lr: 0.0005  max_mem: 9999M
[04/20 12:42:13 d2.utils.events]:  eta: 8:13:20  iter: 12059  total_loss: 0.1595  loss_cls: 0.008003  loss_box_reg: 0.00923  loss_mask: 0.1201  loss_rpn_cls: 0.00513  loss_rpn_loc: 0.003593    time: 3.5962  last_time: 3.7281  data_time: 0.0180  last_data_time: 0.0245   lr: 0.0005  max_mem: 9999M
[04/20 12:43:20 d2.utils.events]:  eta: 8:10:21  iter: 12079  total_loss: 0.1379  loss_cls: 0.006814  loss_box_reg: 0.00885  loss_mask: 0.1115  loss_rpn_cls: 0.00302  loss_rpn_loc: 0.00251    time: 3.5275  last_time: 3.0190  data_time: 0.0181  last_data_time: 0.0193   lr: 0.0005  max_mem: 9999M
[04/20 12:44:26 d2.utils.events]:  eta: 8:08:47  iter: 12099  total_loss: 0.1566  loss_cls: 0.006227  loss_bo

0it [00:00, ?it/s]

[04/20 20:28:28 d2.data.build]: Distribution of instances among all 2 categories:
|  category  | #instances   |  category  | #instances   |
|:----------:|:-------------|:----------:|:-------------|
|   benign   | 5022         | malignant  | 2768         |
|            |              |            |              |
|   total    | 7790         |            |              |
[04/20 20:28:28 d2.data.dataset_mapper]: [DatasetMapper] Augmentations used in inference: [ResizeShortestEdge(short_edge_length=(800, 800), max_size=1333, sample_style='choice')]
[04/20 20:28:28 d2.data.common]: Serializing the dataset using: <class 'detectron2.data.common._TorchSerializedList'>
[04/20 20:28:28 d2.data.common]: Serializing 7790 elements to byte tensors and concatenating them all ...
[04/20 20:28:29 d2.data.common]: Serialized dataset takes 32.03 MiB
WARNING [04/20 20:28:29 d2.engine.defaults]: No evaluator found. Use `DefaultTrainer.test(evaluators=)`, or implement its `build_evaluator` method.


In [17]:
#testing phase

In [18]:
# os.listdir(cfg.OUTPUT_DIR)

In [19]:
# # Inference should use the config with parameters that are used in training
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import ColorMode
import matplotlib.pyplot as plt

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.MODEL.WEIGHTS = "/kaggle/input/models/model_0011999.pth"  # initialize from model zoo
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2  

# # cfg now already contains everything we've set previously. We changed it a little bit for inference:
# cfg.MODEL.WEIGHTS = "../input/d/sashikanthreddy/skin-models/models/model_0019999.pth"  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7 # set a custom testing threshold
predictor = DefaultPredictor(cfg)

In [20]:
# from detectron2.utils.visualizer import ColorMode
from detectron2.utils.visualizer import Visualizer
import cv2
  
im = cv2.imread("/kaggle/input/testing-skin-cancer-dataset/ISIC_0033883_set2.jpg")
# f_name=d["file_name"].split("/")[-1]
outputs = predictor(im)  # format is documented at https://detectron2.readthedocs.io/tutorials/models.html#model-output-format
v = Visualizer(im[:, :, ::-1],
               metadata=region_metadata, 
               scale=1, 
               instance_mode=ColorMode.IMAGE_BW   # remove the colors of unsegmented pixels. This option is only available for segmentation models
)
out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
plt.imshow(out.get_image()[:, :, ::-1])
# plt.imshow(im)
plt.show()
#     cv2.imwrite(f_name,im)
#     cv2.imwrite("predicted_"+f_name,out.get_image()[:, :, ::-1])
# print(f_name)

In [21]:
m="../input/d/sashikanthreddy/skin-models/models/"
for i in os.listdir(m):
    if i.endswith(".pth"):
        cfg.MODEL.WEIGHTS = m+i  # path to the model we just trained
        print(i)
        from detectron2.evaluation import COCOEvaluator, inference_on_dataset
        from detectron2.data import build_detection_test_loader
        evaluator = COCOEvaluator("region_final_test", output_dir="./output")
#         cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
        cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.9  # set a custom testing threshold
        predictor = DefaultPredictor(cfg)
        val_loader = build_detection_test_loader(cfg, "region_final_test")
        print(inference_on_dataset(predictor.model, val_loader, evaluator))
        print("#"*100)

    

In [22]:
pwd

'/kaggle/working'

In [23]:
m="/kaggle/input/skin-cancer-detectron2-models/models/"
for i in os.listdir(m):
    if i.endswith(".pth"):
        cfg.MODEL.WEIGHTS = m+i  # path to the model we just trained
        print(i)
        from detectron2.evaluation import COCOEvaluator, inference_on_dataset
        from detectron2.data import build_detection_test_loader
        evaluator = COCOEvaluator("region_final_test", output_dir="./output")
#         cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
        cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7  # set a custom testing threshold
        predictor = DefaultPredictor(cfg)
        val_loader = build_detection_test_loader(cfg, "region_final_test")
        print(inference_on_dataset(predictor.model, val_loader, evaluator))
        print("#"*100)

    

In [24]:
cfg.MODEL.WEIGHTS = "/kaggle/input/skin-cancer-detectron-models/models/model_0011999.pth"  # path to the model we just trained
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
from detectron2.data import build_detection_test_loader
evaluator = COCOEvaluator("region_final_test", output_dir="./output")
# cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")  # path to the model we just trained
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set a custom testing threshold
predictor = DefaultPredictor(cfg)
val_loader = build_detection_test_loader(cfg, "region_final_test")
print(inference_on_dataset(predictor.model, val_loader, evaluator))
print("#"*100)